In [258]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [259]:
''''
    These are our data sources. Uncomment the one you want to use.
    If you want to use multiple csv's, you'll have to create then combine multiple dataframes,
    Please talk to the group a bit more and we can discuss methodology.
'''
csv_name = "Train_Test_IoT_Fridge.csv"
#csv_name = "Train_Test_IoT_GPS_Tracker.csv"
#csv_name = "Train_Test_IoT_Garage_Door.csv"
#csv_name = "Train_Test_IoT_Modbus.csv"
#csv_name = "Train_Test_IoT_Motion_Light.csv"
#csv_name = "Train_Test_IoT_Thermostat.csv"
#csv_name = "Train_Test_IoT_Weather.csv"

df = pd.read_csv(csv_name)

'''
    Using date and time columns seperately is totally valid.
    Bethany combined them because to_datetime kept throwing errors when they were seperate
    If this single time & date messes with your config, please comment out the drop statement and
         deal with the consequences at your leisure.
'''

df['date_time'] = df[['date', 'time']].agg(' '.join, axis=1)
df.date_time = pd.to_datetime(df.date_time)
df2 = df.drop(['date', 'time'], axis=1)
#print(df2.info())
if df.label.count() == df2.label.count():
  print("\n Counts are good. Date_time object is created.")
else:
  print("\n Something went wrong, Time to troubleshoot")

<ipython-input-259-bd3ff710d288>:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.date_time = pd.to_datetime(df.date_time)



 Counts are good. Date_time object is created.


In [260]:
df2.date_time = pd.to_numeric(df.date_time)

In [261]:
'''
    Don't change this code. It handles some specific object related issues in
          Garage_Door
          Motion_Light
'''
df2 = df2.map(lambda x: x.strip() if isinstance(x, str) else x)
if csv_name == 'Train_Test_IoT_Garage_Door.csv':
  df2 = df2.replace({"true": 1, "false": 0})
  df2.sphone_signal = pd.to_numeric(df2.sphone_signal)
df2.info()

"""
    These are all the categories of "Type" you can process.
          normal
          backdoor
          ddos
          injection
          password
          ransome
          xss
          scanning

      Not all of these catagories exist in each dataset.
      If it doesn't exist, you'll get 'Attacks: 0'.
      Pick another attack.
"""
attack = 'backdoor'
attack_df = df2[df2.type== attack]
#print(attack_df.head())
attack_count = attack_df.type.count()
print(f'\nAttacks: ', attack_count)
if attack_count != 0 :
  half_count = int(attack_count/2)
  print(f'Train subset should be: ', half_count)
else:
  print("\n This type of attack does not exist! Pick another.")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39944 entries, 0 to 39943
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fridge_temperature  39944 non-null  float64
 1   temp_condition      39944 non-null  object 
 2   label               39944 non-null  int64  
 3   type                39944 non-null  object 
 4   date_time           39944 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.5+ MB

Attacks:  5000
Train subset should be:  2500


In [262]:
'''
    This code creates a boolean for whether or not the instance contains the specific attack
        that you're looking for
'''
df_encoded = pd.get_dummies(attack_df)

if df_encoded.shape[0] == attack_count:
  print(f"\n You didn't lose any rows. All Good \n")
  print(f" Current count is ", df_encoded.shape[0])
else:
  print(f"\n Something went wrong. Current count is  {df_encoded.shape[0]} and should be {attack_count}")


 You didn't lose any rows. All Good 

 Current count is  5000


In [263]:
'''
 The .sample() function takes a random sample without replacement.
 N = half the attack count unless attack count == 0, then it will do nothing.
Then the dataframe is split again,:
       train_bd_data --> drops columns Type and Label
       train_bd_labels --> only columns Type and Label
'''
if attack_count != 0 :
  train_bd = df_encoded.sample(n=half_count)
  #train_bd.head()
  #train_bd.info()
  train_bd_data = train_bd.drop(columns=[f"type_{attack}", "label"])
  train_bd_labels = train_bd[[f"type_{attack}", "label"]]

  '''
    Verification that you still have retained all your data.
    Uncomment if you want to check columns.
'''
#train_bd_data.info()
#train_bd_labels.info()
if train_bd_data.shape[0] == train_bd_labels.shape[0]:
  print(f"\n You didn't lose any rows. All Good \n")



 You didn't lose any rows. All Good 



In [264]:
'''
    Encode->  Should output a 1D matrix: (attack count, 1)
    Scale -> Should output 1D matrix: (half_count, # of left over columns)
    Reshape -> Should output 3D matrix: (half_count, # of left over columns, 1)
'''
enc = OneHotEncoder(handle_unknown='ignore')
train_bd_labels = enc.fit_transform(train_bd_labels.values.reshape(-1, 1)).toarray()
print(train_bd_labels.shape)


# scale
scaler = StandardScaler()
train_bd_data = scaler.fit_transform(train_bd_data)
print(train_bd_data.shape)


# reshape
train_bd_data = train_bd_data.reshape(train_bd_data.shape[0], train_bd_data.shape[1], 1)
print(train_bd_data.shape)

(5000, 1)
(2500, 4)
(2500, 4, 1)
